<a href="https://colab.research.google.com/github/Catatau52/Carteira_Quant/blob/main/Etapa_1_carregando_data_fintz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyarrow

In [ ]:
import requests
import pandas as pd
import os
import urllib.request
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import requests as req

# URL_BASE = 'https://api.fintz.com.br'
# HEADERS = { 'X-API-Key': 'your-key' }
# PARAMS = { 'ticker': 'BBAS3', 'dataInicio': '2023-04-01' }

# endpoint = URL_BASE + '/bolsa/b3/avista/cotacoes/historico'
# res = req.get(endpoint, headers=HEADERS, params=PARAMS)
# print(res.json())

In [ ]:
#https://docs.fintz.com.br/endpoints/bolsa/
class dados_fintz:

    def __init__(self, caminho_dados):

        #self.chave_api = os.getenv("API_FINTZ")
        self.chave_api = 'your-key'
        self.headers = {'accept': 'application/json',
                        'X-API-Key': self.chave_api}
        os.chdir(caminho_dados)

    def cdi(self):

        response = requests.get('https://api.fintz.com.br/taxas/historico?codigo=12&dataInicio=2000-01-01&ordem=ASC',
                                headers=self.headers)

        cdi = pd.DataFrame(response.json())

        cdi = cdi.drop(["dataFim", 'nome'], axis = 1)

        cdi.columns = ['data', 'retorno']

        cdi['retorno'] = cdi['retorno']/100

        cdi.to_parquet('cdi.parquet', index = False)


    def ibov(self):

        response = requests.get('https://api.fintz.com.br/indices/historico?indice=IBOV&dataInicio=2000-01-01',headers=self.headers)
        #response = requests.get('https://api.fintz.com.br/indices/historico?indice=IDIV&dataInicio=2000-01-01',headers=self.headers)

        df = pd.DataFrame(response.json())

        df = df.sort_values('data', ascending=True)

        df.columns = ['indice', 'data', 'fechamento']

        df = df.drop('indice', axis = 1)

        df.to_parquet('ibov.parquet', index = False)

    def pegar_cotacoes(self):


        response = requests.get(f'https://api.fintz.com.br/bolsa/b3/avista/cotacoes/historico/arquivos?classe=ACOES&preencher=true',
                                headers=self.headers)

        link_download = (response.json())['link']

        urllib.request.urlretrieve(link_download, f"cotacoes.parquet")

        df = pd.read_parquet('cotacoes.parquet')

        colunas_pra_ajustar = ['preco_abertura', 'preco_maximo', 'preco_medio', 'preco_minimo']

        for coluna in colunas_pra_ajustar:

            df[f'{coluna}_ajustado'] = df[coluna] * df['fator_ajuste']

        df['preco_fechamento_ajustado'] = df.groupby('ticker')['preco_fechamento_ajustado'].transform('ffill')

        df = df.sort_values('data', ascending=True)

        df.to_parquet('cotacoes.parquet', index = False)


    def pegando_arquivo_contabil(self, demonstracao = False, indicadores = False, nome_dado = ''):

        if demonstracao:

            try:

                response = requests.get(f'https://api.fintz.com.br/bolsa/b3/tm/demonstracoes/arquivos?item={nome_dado}',
                                        headers=self.headers)

            except:

                print("Demonstração não encontrada!")
                exit()

            link_download = (response.json())['link']
            urllib.request.urlretrieve(link_download, f"{nome_dado}.parquet")

        elif indicadores:

            try:

                response = requests.get(f'https://api.fintz.com.br/bolsa/b3/tm/indicadores/arquivos?indicador={nome_dado}',
                                        headers=self.headers)

            except:

                print("Indicador não encontrado!")
                exit()

            link_download = (response.json())['link']
            urllib.request.urlretrieve(link_download, f"{nome_dado}.parquet")

        else:

            print("Escolha uma demonstração ou indicador.")


if __name__ == "__main__":

    lendo_dados = dados_fintz(caminho_dados=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br')

    lendo_dados.pegar_cotacoes()

    lista_demonstracoes = ['PatrimonioLiquido','LucroLiquido12m','LucroLiquido','ReceitaLiquida','ReceitaLiquida12m','DividaBruta','DividaLiquida','Disponibilidades','Ebit','Ebit12m','TotalAcoes']
    #lista_demonstracoes = ['TotalAcoes']
    lista_indicadores = ['ROE', 'ROIC', 'LPA', 'L_P', 'P_L', 'EV','EBIT_EV', 'EV_EBIT','ValorDeMercado']
    #lista_indicadores = ['P_L','L_P']

    for demonstracao in lista_demonstracoes:

        print(demonstracao)

        lendo_dados.pegando_arquivo_contabil(demonstracao=True, nome_dado = demonstracao)

    for indicador in lista_indicadores:

        print(indicador)

        lendo_dados.pegando_arquivo_contabil(indicadores=True, nome_dado = indicador)


    lendo_dados.cdi()
    lendo_dados.pegar_cotacoes()
    lendo_dados.ibov()



PatrimonioLiquido
LucroLiquido12m
LucroLiquido
ReceitaLiquida
ReceitaLiquida12m
DividaBruta
DividaLiquida
Disponibilidades
Ebit
Ebit12m
TotalAcoes
ROE
ROIC
LPA
L_P
P_L
EV
EBIT_EV
EV_EBIT
ValorDeMercado


In [ ]:

class market_premium():

    def __init__(self, caminho_salvar_arquivo = '.', caminho_dados='.'):

        self.caminho_salvar_arquivo = caminho_salvar_arquivo
        self.caminho_dados = caminho_dados

    def calculando_premio(self):

        cdi = pd.read_parquet(f'{self.caminho_dados}/cdi.parquet')
        cdi['cota'] = (1 + cdi['retorno']).cumprod() - 1
        ibov = pd.read_parquet(f'{self.caminho_dados}/ibov.parquet')

        ibov_datas = ibov.sort_values('data', ascending = True)
        ibov_datas = ibov_datas.assign(year = pd.DatetimeIndex(ibov_datas['data']).year)
        ibov_datas = ibov_datas.assign(month = pd.DatetimeIndex(ibov_datas['data']).month)
        datas_final_mes = ibov_datas.groupby(['year', 'month'])['data'].last()
        dias_final_de_mes = datas_final_mes.to_list()

        ibov = ibov[ibov['data'].isin(dias_final_de_mes)]
        cdi = cdi[cdi['data'].isin(dias_final_de_mes)]
        ibov['retorno_ibov'] = ibov['fechamento'].pct_change()
        cdi['retorno_cdi'] = cdi['cota'].pct_change()
        ibov['data'] = ibov['data'].astype(str)
        cdi['data'] = cdi['data'].astype(str)

        df_dados_mercado = pd.merge(ibov, cdi, how = 'inner', on = "data")
        df_dados_mercado['mkt_premium'] = (1 + df_dados_mercado['retorno_ibov'])/(1 + df_dados_mercado['retorno_cdi']) - 1
        df_dados_mercado = df_dados_mercado.dropna()
        df_dados_mercado = df_dados_mercado[['data', 'mkt_premium']]
        df_dados_mercado['data'] = pd.to_datetime(df_dados_mercado['data']).dt.date

        df_dados_mercado.to_parquet(f'{self.caminho_salvar_arquivo}/market_premium.parquet', index = False)


if __name__ == "__main__":

    beta = market_premium(caminho_salvar_arquivo=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br',
                          caminho_dados=r'/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br')

    beta.calculando_premio()

In [ ]:
df = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/Carteira_Quant/base_dados_br/premios_risco_br/market_premium.parquet')
df

,data,mkt_premium
0,2000-02-29,-0.464910
1,2000-03-31,-0.331790
2,2000-04-28,-0.332249
3,2000-05-31,-0.244952
4,2000-06-30,-0.070835
...,...,...
283,2023-09-29,-0.003285
284,2023-10-31,-0.039620
285,2023-11-30,0.114461
286,2023-12-28,0.044325
